# Исследовательский проект по НИС «Анализ данных в Python»
<div style="text-align: right">
    Проект подготовлен студентами БПИ228
    <br>
    Лысиным Кириллом и Гакал Анжеликой
</div>

## Часть 1.

## Часть 2.

In [3]:
#import requests
from selenium import webdriver
import pandas as pd
import time
from bs4 import BeautifulSoup
import os.path


    
driver = webdriver.Chrome()

with open('visited_links.txt', 'w+') as f:
    visited_links = set(f.readlines())


def get_item_info(url):
    # Можно сделать словарь вместо массива для более удобного обращения к элементам
    item_info = []
    info_dict = {}
    
    # Тут должны вызываться методы, возвращающие строки с нужными значениями, которые будут добавляться в item_info
    soup = get_soup(url)
    if(soup == None):
        return
    
    general_info = str(soup.findAll('meta', {'property':"og:description"})[0])

    #через список
    item_info.append(get_item_name(general_info))
    item_info.append(get_item_price(general_info))
    item_info.append(get_item_category(general_info))
    item_info.append(get_item_region(soup))
    item_info.append(get_item_placement_time(soup))
    item_info.append(get_seller_name(soup))
    item_info.append(get_number_of_ads(soup))
    item_info.append(get_seller_rating(soup))
    item_info.append(get_number_of_views(soup))
    item_info.append(get_number_of_images(soup))
    item_info.append(url)
    
    #через словарь
    info_dict['Name'] = get_item_name(general_info)
    info_dict['Price (in rubles)'] = get_item_price(general_info)
    info_dict['Category'] = get_item_category(general_info)
    info_dict['Region'] = get_item_region(soup)
    info_dict['Time/date of placement'] = get_item_placement_time(soup)
    info_dict['Seller\'s name'] = get_seller_name(soup)
    info_dict['Number of seller\'s ads'] = get_number_of_ads(soup)
    info_dict['Seller rating'] = get_seller_rating(soup)
    info_dict['Number of views'] = get_number_of_views(soup)
    info_dict['Number of images'] = get_number_of_images(soup)
    info_dict['Link'] = url
    
    return item_info, info_dict

def get_soup(url, counter = 0):

    time.sleep(1)
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    # Если не удалось сразу получить страницу, то программа пробует еще несколько раз
    except:
        if(counter < 3):
            return get_soup(url, counter + 1)
    # Если после несольких попыток так и не удалось считать содержимое страницы, то этот метод вернет None

def get_item_name(info):
    try:
        name = info[15:info.find('–') - 1]
        return name
    except:
        return None

def get_item_price(info):
    if not('Цена: ' in info):
        return 0
    price = float(info[info.find('Цена: ') + 6:info.find('руб.') - 1].replace(' ', ''))
    return price

def get_item_category(info):
    try:
        category = info[info.find('раздела «') + 9:info.find('».')]
        return category
    except:
        return None

def get_item_region(soup):
    try:
        region = list(soup.findAll('span', class_='sc-cOxWqc hNYaaO'))[0].text
        return region
    except:
        return None

def get_item_placement_time(soup):
    try:
        time_of_placement = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))
        return time_of_placement[-1].text
    except:
        return None

def get_seller_name(soup):
    try:
        name_of_seller = list(soup.findAll('span', {'data-test-component':"UserNameClick"}))[0].text
        name_of_seller = name_of_seller[:name_of_seller.find('(') - 1]
        return name_of_seller
    except:
        return None

def get_number_of_ads(soup):
    try:
        n = list(soup.findAll('span', {'data-test-component':"UserNameClick"}))[0].text
        n = int(n[n.find('(')+1:n.find(' объявл')])
        return n
    except:
        return None

def get_seller_rating(soup):
    try:
        rating = soup.find('span', class_="sc-cOxWqc eDoIYl")
        return float(rating.text.replace(',', '.')) if rating != None else None
    except:
        return None

def get_number_of_views(soup):
    try:
        number_of_views = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))
        return int(number_of_views[-2].text)
    except:
        return None

def get_number_of_images(soup):
    try:
        images = list(soup.findAll('img', class_="sc-fBnnfK"))
        return len(images)
    except:
        return None

def get_links(scroll_count):
    
    url = r'https://youla.ru/'
    driver.get(url)
    time.sleep(1)
    
    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    links = [str(i.a) for i in soup.findAll('span', class_="sc-llGDqb sc-gqgnwQ fEAASo hZGRky")]
    links = [r'https://youla.ru' + s[s.find(r'href="') + 6 : s.find(r'" rel')] for s in links]
    return links




for i in range(100):
    items = []
    links = get_links(1)
    print("Processing data. Please wait...")
    for i in range(len(links)):
        if(int(i * 100/len(links))//5 > int((i - 1) * 100/len(links))//5):
            print(f'\t{((i * 100/len(links))//5)*5}%')
        link = links[i]
        if(link in visited_links):
            continue
        items.append(get_item_info(link)[1])
        visited_links.add(link)
    print('\t100.0%')

    df = pd.DataFrame(items)
    df = df.set_index('Name')

    if(os.path.isfile('items.xlsx')):
        prev_df = pd.read_excel('items.xlsx', index_col = 0)
        df = pd.concat([prev_df, df])

    df.to_excel('items.xlsx')



with open('visited_links.txt', 'a') as f:
    for line in visited_links:
        f.write(f"{line}\n")



driver.quit()
df

Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55

	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. P

	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please w

,Price (in rubles),Category,Region,Time/date of placement,Seller's name,Number of seller's ads,Seller rating,Number of views,Number of images,Link
Name,,,,,,,,,,
Детское питание,0.0,Детские товары,Москва,Сегодня в 21:18,Екатерина В.,6.0,5.0,117,0,https://youla.ru/moskva/detskie/kormlenie-pita...
Крсивый пес Шерман пропадает в приюте,0.0,Животные,Москва,Сегодня в 21:18,Елена Потапова,52.0,NaN,53,6,https://youla.ru/moskva/zhivotnye/sobaki/krsiv...
Футляр для телефона.,0.0,Телефоны и планшеты,Балашиха,Сегодня в 21:18,Элеонора К.,45.0,5.0,76,0,https://youla.ru/balashiha/smartfony-planshety...
Туфли стильные на небольшой танкетке р. 36,0.0,Женский гардероб,Москва,Сегодня в 21:18,МП Е.,224.0,5.0,206,0,https://youla.ru/moskva/zhenskaya-odezhda/obuv...
Водолазка прозрачная,0.0,Женский гардероб,Лобня,Сегодня в 21:18,Эля Я.,32.0,5.0,355,0,https://youla.ru/lobnya/zhenskaya-odezhda/bluz...
...,...,...,...,...,...,...,...,...,...,...
"Cadillac CTS, 2011",828000.0,Легковые автомобили,Москва,Сегодня в 12:41,Менеджер,182.0,NaN,3,14,https://youla.ru/moskva/auto/s-probegom/cadill...
Самсунг а 40,8000.0,Телефоны и планшеты,Москва,Сегодня в 8:59,An C.,2.0,NaN,2,2,https://youla.ru/moskva/smartfony-planshety/sm...
Новая корзика,250.0,Для дома и дачи,Москва,1 января,Татьяна Б.,36.0,5.0,12,3,https://youla.ru/moskva/dom-dacha/oformlenie-i...


In [1]:
import pandas as pd
df = pd.DataFrame({
    'city':['Austin', 'Denver', 'Springfield', 'Mendocino'], 
    'branch_id':[10, 20, 30, 47],
    'revenue':[100, 83, 4, 200]})

df.to_excel('items_1.xlsx')